# 计算区域排水能力

In [8]:
import arcpy
import os
from arcpy.sa import *

In [11]:
# 设置环境参数
arcpy.env.workspace = r"C:\Users\Runker\Desktop\DEM_test\drainwater.gdb"
arcpy.env.overwriteOutput = True
# 设置单进程模式
arcpy.env.parallelProcessingFactor = "1"
# 输入数据
dem = "DEM"
geology_type = "DZ"
rivers_shp = "river"
canals_shp = "vally"

In [12]:
# 步骤1：填平DEM数据中的凹陷
filled_dem = Fill(dem)
filled_dem.save("filled_dem")

In [13]:
# 步骤2：计算坡度
slope = Slope(filled_dem, "DEGREE")
slope.save("slope")

In [14]:
# 步骤3：计算流向
flow_direction = FlowDirection(filled_dem)
flow_direction.save("flow_direction")

In [15]:
# 步骤4：计算流积量
flow_accumulation = FlowAccumulation(flow_direction)
flow_accumulation.save("flow_accumulation")

In [ ]:
# 步骤5：计算相对位置
neighborhood = NbrRectangle(101, 101, "CELL")
local_min = FocalStatistics(Raster(dem), neighborhood, "MINIMUM")
local_max = FocalStatistics(Raster(dem), neighborhood, "MAXIMUM")
normalized_local = (Raster(dem) - local_min) / (local_max - local_min)
position = Reclassify(normalized_local, "Value", 
                      RemapRange([[0, 0.2, 1], [0.2, 0.7, 2], [0.7, 1, 3]]))
position.save("position")

In [16]:
# 步骤6：合并河流和沟渠图层
arcpy.management.Merge([rivers_shp, canals_shp], "combined_features")

# 步骤7：计算到河流和沟渠的距离
distance_to_river_canal = EucDistance(in_source_data="combined_features",cell_size="5")
distance_to_river_canal.save("distance_to_river_canal")

In [18]:
# 步骤8：标准化所有因子
def normalize_raster(raster):
    min_val = float(arcpy.GetRasterProperties_management(raster, "MINIMUM").getOutput(0))
    max_val = float(arcpy.GetRasterProperties_management(raster, "MAXIMUM").getOutput(0))
    return (Raster(raster) - min_val) / (max_val - min_val)

normalized_slope = normalize_raster("slope")
normalized_flow_accumulation = normalize_raster("flow_accumulation")
normalized_distance = normalize_raster("distance_to_river_canal")
normalized_position = (Raster("position") - 1) / 2

In [21]:
# 标准化地质类型
geology_remap = RemapValue([
    [1, 10], [2, 50], [3, 30], [4, 90], [5, 60],
    [6, 80], [7, 20], [8, 85], [9, 75]
])
normalized_geology = Reclassify(geology_type, "Value", geology_remap) / 100


In [23]:
# 步骤9：计算排水能力
drainage_capacity = (0.3 * normalized_slope + 
                     0.2 * normalized_flow_accumulation + 
                     0.15 * normalized_geology + 
                     0.1 * (1 - normalized_distance) +  # 距离越近，排水能力越强
                     0.25 * normalized_position)
drainage_capacity.save("drainage_capacity")

In [24]:
# 步骤10：分类排水能力
classified_drainage = Reclassify(drainage_capacity, "Value",
                                 RemapRange([[0, 0.25, 1], [0.25, 0.5, 2], 
                                             [0.5, 0.75, 3], [0.75, 1, 4]]))
classified_drainage.save("classified_drainage_capacity")

print("排水能力计算完成。")

排水能力计算完成。
